In [ ]:
# Copiando os dados gerados anteriormente 
# Pesos do Portfolio de mercado
mu_t_df = mkt_portfolio_weights.copy()

# Retorno das ações
R_t_df =  stocks_returns.copy()

#Retorno do portfólio de mercado
#Iloc[1:] para iniciar as series em 2004-01-05 e eliminar os valores NaN decorrentes do calculo do retorno. 

Y = tf.convert_to_tensor(mkt_return.iloc[1:], dtype=tf.float32)              # Retorno do portfólio de mercado
X = tf.convert_to_tensor(mu_t_df.shift(1).iloc[1:].values, dtype=tf.float32) # Pesos de mercado

# Prepare stock returns data (should align with training samples)
stock_returns = tf.convert_to_tensor(R_t_df.iloc[1:].values, dtype=tf.float32)


from sklearn.model_selection import train_test_split

indices = np.arange(len(X))
train_idx, test_idx = train_test_split(indices, test_size=0.1, shuffle=False)

# Split datasets
X_train = tf.convert_to_tensor(X.numpy()[train_idx], dtype=tf.float32)
X_test  = tf.convert_to_tensor(X.numpy()[test_idx], dtype=tf.float32)

Y_train = tf.convert_to_tensor(Y.numpy()[train_idx] , dtype=tf.float32)
Y_test  = tf.convert_to_tensor(Y.numpy()[test_idx], dtype=tf.float32)

stock_returns_train = tf.gather(stock_returns, train_idx)
stock_returns_test = tf.gather(stock_returns, test_idx)


input_dim =  X.shape[1] #100



class PortfolioModel(tf.keras.Model):
    def __init__(self, network):
        super().__init__()
        self.network = network
        #self.stock_returns = tf.convert_to_tensor(stock_returns, dtype=tf.float32)
        
    def train_step(self, data):
        X, y_true = data
        
        with tf.GradientTape(persistent=True) as tape:
            
            # Compute gradient of log(G) with respect to INPUTS (X)
            with tf.GradientTape() as inner_tape:
                inner_tape.watch(X)
                G = self.network(X)
                log_G = tf.math.log(G + 1e-7)
                
            grad_log_G_X = inner_tape.gradient(log_G, X)
            
            # Calcula os pesos do portfólio
            sum_term = tf.reduce_sum(X * grad_log_G_X, axis=1, keepdims=True)
            pi_weights = X * (grad_log_G_X + 1 - sum_term)
            port_returns = tf.reduce_sum(tf.reduce_sum(pi_weights*y_true, axis=1))
            
            mkt_return = tf.reduce_sum(tf.reduce_sum(X*y_true, axis=1))
                        
            # Custom loss calculation
            loss = -(port_returns - mkt_return)
            
        # Compute gradients and update weights
        trainable_vars = self.network.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        del tape      
        return {'loss': loss}
    


    # Build and train model
l2_lambda = 0.0001
base_network = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(input_dim,)),
    tf.keras.layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)),
    tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)),
    tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)),
    tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)),
    tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)),
    tf.keras.layers.Dense(1,  activation='softmax',bias_initializer=tf.keras.initializers.Constant(0.5)) #Softplus para garantir que o output (G) seja >0.
])

model = PortfolioModel(base_network)
model.compile(optimizer=tf.keras.optimizers.Adam())
model.fit(x=tf.data.Dataset.from_tensor_slices((X_train, Y_train)).batch(32), epochs=50)



base_network = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(input_dim,)),
    tf.keras.layers.Dense(256, kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(alpha=0.1),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(128),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(64),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(32),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(32),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(1, activation='softmax', bias_initializer=tf.keras.initializers.Constant(0.25))
])

model = PortfolioModel(base_network)
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001))
history = model.fit(
    x=tf.data.Dataset.from_tensor_slices((X_train, Y_train)).batch(64),
    epochs=50,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)



# Compute gradients for portfolio weights (critical step)
with tf.GradientTape() as tape:
    tape.watch(X_test)
    G_test = model.network(X_test)  
    log_G_test = tf.math.log(G_test)

grad_log_G_X_test = tape.gradient(log_G_test, X_test).numpy()





# Calculate portfolio weights (same formula as training)
sum_term_test = tf.reduce_sum(X_test * grad_log_G_X_test, axis=1, keepdims=True)
pi_weights_test = X_test * (grad_log_G_X_test + 1 - sum_term_test)

# Calculate portfolio returns
portfolio_returns_test = tf.reduce_sum(pi_weights_test * stock_returns_test, axis=1)

# Compare with true market returns (Y_test)
#portfolio_performance = tf.reduce_mean(portfolio_returns_test - tf.squeeze(Y_test))

# mkt Return
mkt_ret_test = tf.reduce_sum(X_test*Y_test,axis=1)



plt.figure(figsize=(12, 5))

plt.plot(mkt_return_t_df.iloc[-539:].index,((1+portfolio_returns_test.numpy()).cumprod()-1),label='Retorno do portfolio gerado pela NN') 
plt.plot(mkt_return_t_df.iloc[-539:].index, ((1+mkt_ret_test.numpy()).cumprod()-1),label='Retorno do portfólio de Mercado')

plt.legend()
plt.grid(True)
plt.show()



